# Conjugate Gradients with operators

Written by H. Tagare 2/8/2024

The operators are defined in the folder optlib
The CG algorithm is defined in the folder c_grad
Both have to imported to get everything to work

In [4]:
import numpy as np
import optlib.operators as op
import optlib.c_grad as cg
import importlib
importlib.reload(cg)

<module 'optlib.c_grad' from 'C:\\Users\\hdtag\\rpt\\2024\\c_grad\\optlib\\c_grad.py'>

### Test 1: Simple test without noise
#### The operator A is (MO+R). Task is to recover a vector. No regularization on CG

In [5]:
M=op.matrix_op(np.array(((1, 2, 3),(0, 1, 0),(2, 3,1))))
O=op.scalar_prod_op(2.0)
R=op.scalar_prod_op(0.1)
A=op.add_op(op.composite_op(M,O),R) 

x=np.array((1,0.5,1)).reshape(-1,1) #Input x
y=A.forward(x) #Create noiseless output by applying A
print(f"x={x}\n y={y}")

x0=np.zeros_like(x) #Initial value for CG
# x,flag=cg.c_grad(y,A,x0) #Apply CG
x,flag=cg.solve_lin_cg(y,A,x0)
print(f"x={x} \n flag={flag}")

x=[[1. ]
 [0.5]
 [1. ]]
 y=[[10.1 ]
 [ 1.05]
 [ 9.1 ]]
CG: step=3 res_norm=6.937076823812195e-12 
x=[[1. ]
 [0.5]
 [1. ]] 
 flag=False


### Test 2: x is matrix, y contains noise, CG uses regularizer

In [6]:
importlib.reload(cg)
M=op.matrix_op(np.array(((1, 2, 3),(0, 1, 0),(2, -3,1))))
O=op.scalar_prod_op(2.0)
R=op.scalar_prod_op(0.1)

A=op.add_op(op.composite_op(M,O),R)

x=np.transpose(np.array(((1,0.5,1),(0,1,0))))
y=A.forward(x)
y=y+0.1*np.random.normal(size=y.shape)
print(f"x={x}\n y={y}")

x0=np.zeros_like(x)
x,flag=cg.solve_L2_min(y,A,x0,B=R,f_tol=1e-6)
print(f"x={x} \n flag={flag}")


x=[[1.  0. ]
 [0.5 1. ]
 [1.  0. ]]
 y=[[10.1045605   4.01415818]
 [ 1.01948538  2.20375285]
 [ 2.93900117 -6.20903933]]
CG: step=3 res_norm=1.410427453446521e-13 
x=[[ 0.91726717  0.04643509]
 [ 0.48546923  1.04940612]
 [ 1.03940376 -0.04683   ]] 
 flag=False


### CG with fft

In [8]:
importlib.reload(cg)
x=np.random.normal(size=(4,4))
print(x)
A=op.real_fftn_op()
y=A.forward(x)
y=y+0.2*np.random.normal(size=y.shape)
#print(f"x={x}\n y={y}")
#print(f"x={x}")

x0=np.zeros_like(x)
R=op.scalar_prod_op(0.1)
x,_=cg.solve_L2_min(y,A,x0,B=R)
print(x)

[[ 1.04071935 -0.61788709 -0.32255295 -0.88875921]
 [ 2.19764072  1.33715726  0.38545432 -1.00781026]
 [ 0.45016272  0.08440863 -2.33684131  0.68688993]
 [ 0.3981621   1.49109557  0.01734395 -0.29766602]]
CG: step=1 res_norm=4.3709524189038457e-16 
[[ 1.09623369 -0.60449197 -0.23753085 -0.87536409]
 [ 2.12208495  1.32236843  0.41651987 -0.97397677]
 [ 0.45812757  0.09290777 -2.33046873  0.69538907]
 [ 0.32260634  1.52492906  0.0484095  -0.31245486]]
